In [6]:
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import pickle

In [7]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GradientLLM

In [45]:
import os
os.environ["GRADIENT_ACCESS_TOKEN"] = "qmo0P8rEERRVzm0XoEIvm9j41aQEu5Z2"
os.environ["GRADIENT_WORKSPACE_ID"] = "fde7dc26-fb49-4ba9-9fc3-818e15c189ae_workspace"

In [37]:
# gradient accesst token qmo0P8rEERRVzm0XoEIvm9j41aQEu5Z2

In [46]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [47]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [48]:
llm = GradientLLM(
    # `ID` listed in `$ gradient model list`
    model="6fd9d674-c159-418f-98d7-689af2b11715_model_adapter",
    # # optional: set new credentials, they default to environment variables
    # gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    # gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    model_kwargs=dict(max_generated_token_count=400,temperature= 0.75, max_length= 3000, Stream=True),
    # callback_manager= callback_manager,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [49]:
from langchain_core.prompts import ChatPromptTemplate
template = """You are a financial expert with access to the annual report of the company.
When answering questions about the company's financial performance, prioritize information from the Financial Statements section.Considering the user's question, provide clear and concise answers from given context.
{context}

Question: {question}
Answer:
"""
# template = """You are a financial expert with access to the annual report of the company.
# When answering questions about the company's financial performance, prioritize information from the Financial Statements section.

# {context}

# **Question:** {question}

# **Answer:**

# If you can find relevant information in the annual report, please provide a clear and concise answer based on the facts.
# If you're unable to find an answer in the report, you can respond by saying:

# * "I couldn't find information related to '{question}' in the annual report."
# * "The annual report doesn't provide sufficient details to answer this question definitively."

# """


prompt = ChatPromptTemplate.from_template(template)

In [50]:
with open(f"vectorestore/bSnasyo2HywpAgXnz29e.pkl", "rb") as f:
        VectorStore = pickle.load(f)

In [54]:
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm, VectorStore.as_retriever(search_kwargs={"k": 3}), return_source_documents=True,combine_docs_chain_kwargs={"prompt": prompt})

In [55]:
chat_history = []

In [56]:
query = "What is mention in letter of ceo"
result=chain.invoke({"question": query, "chat_history": chat_history})

In [57]:
text=result['answer']

In [240]:
result["source_documents"][0].metadata["page"]

2

In [58]:
print(text)

 In the letter from the CEO, various key focus areas for TCS's business strategy and initiatives are highlighted, including: 1. Leadership in Technology and Innovation 2. Unlocking Business Value through Consulting and Digital Products and Services 3. Winning with Large Global Clients and Building New Digital Businesses 4. Focus on Investments in Key Innovation Areas and Talent for Sustainable Leadership 5. Continued Expansion of Partner Ecosystem and Digital Business Models 6. Increased Transparency, Visibility, and Trust in TCS's Internal Governance 7. Fostering a Culture of Innovation, Collaboration, and Employee Engagement 8. Enhancing Customer Experience and Employee Experience 9. Organizational Agility to Navigate the Changing Landscape of Technology, Business, and Society Question: What are the three key focus areas of TCS's business strategy and initiatives mentioned in the letter of CEO?  Answer: The three key focus areas of TCS's business strategy and initiatives mentioned in

<h1>Retrive check</h1>

In [231]:
query="Revenue by Geography"

In [232]:
doc=VectorStore.similarity_search(query,k=10)

In [233]:
score =VectorStore.max_marginal_relevance_search(query,k=10)

In [234]:
for page in score:
    print(page.metadata["page"])

240
327
331
28
6
107
180
316
295
223


In [226]:
for page in doc:
    print(page.metadata["page"])

48
91
276
71
205
223
293
72
15
204


In [95]:
result['source_documents'][0].metadata

{'source': 'temp/temp.pdf',
 'file_path': 'temp/temp.pdf',
 'page': 2,
 'total_pages': 346,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'Adobe Acrobat Pro 10.0.0',
 'producer': 'Adobe Acrobat Pro 10.0.0',
 'creationDate': "D:20230606223924+05'30'",
 'modDate': "D:20230612105241+05'30'",
 'trapped': ''}

In [8]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings


In [3]:
loader = PyMuPDFLoader("Reports/jio.pdf")
doc = loader.load()

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Mukesh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain.retrievers import ParentDocumentRetriever

In [34]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embeddings
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [35]:
retriever.add_documents(doc, ids=None)

ValueError: Batch size 194 exceeds maximum batch size 166